In [5]:
from constants import train_trials
from functions import load_data, run_trial

In [6]:
dfs = load_data(tcks=("AAPL", "DELL", "FORD", "IBM", "MACYS", "SP500"))

In [3]:
# Replicate trials from the paper
mapes = {}
dpas = {}
preds = {}

# Paper constants (adjust as needed)
latency = 10
n_states = 4

for tck, trials in train_trials.items():
    mapes[tck] = {}
    dpas[tck] = {}
    for i, trial in trials.items():
        # TODO: Write predictions to CSV file
        results, _ = run_trial(
            dfs,
            tck=tck, train_period=trial["train"], test_period=trial["test"],
            latency=latency, n_states=n_states
        )
        mapes[tck][i] = results["MAPE"]
        dpas[tck][i] = results["DPA"]

Training HMM on train period ('2003-02-10', '2004-09-10')
Training Complete
Generating Predictions for test period ('2004-09-13', '2005-01-21')


100%|██████████| 92/92 [01:11<00:00,  1.29it/s]


Predictions Generated
AAPL trial with latent states = 4, context window size = 10
MAPE = 0.01556451275223328, DPA = 0.5652173913043478
--------------------------------------------------------------------------------
Training HMM on train period ('2003-02-10', '2004-09-10')
Training Complete
Generating Predictions for test period ('2004-10-13', '2005-01-21')


100%|██████████| 70/70 [00:53<00:00,  1.31it/s]


Predictions Generated
AAPL trial with latent states = 4, context window size = 10
MAPE = 0.015598935045108387, DPA = 0.6142857142857143
--------------------------------------------------------------------------------
Training HMM on train period ('2021-01-04', '2022-01-03')
Training Complete
Generating Predictions for test period ('2023-01-03', '2023-06-30')


...

Predictions Generated
IBM trial with latent states = 4, context window size = 10
MAPE = 0.012885398684269076, DPA = 0.3695652173913043
--------------------------------------------------------------------------------
Training HMM on train period ('2021-01-04', '2023-01-03')
Training Complete
Generating Predictions for test period ('2023-01-04', '2023-07-11')


100%|██████████| 129/129 [01:39<00:00,  1.30it/s]


Predictions Generated
IBM trial with latent states = 4, context window size = 10
MAPE = 0.008041129874531263, DPA = 0.46511627906976744
--------------------------------------------------------------------------------
Training HMM on train period ('2021-01-04', '2022-01-03')
Training Complete
Generating Predictions for test period ('2023-01-03', '2023-07-11')


100%|██████████| 130/130 [01:38<00:00,  1.31it/s]

Predictions Generated
DELL trial with latent states = 4, context window size = 10
MAPE = 0.01565943824254912, DPA = 0.6307692307692307
--------------------------------------------------------------------------------


In [ ]:
# Experiment 1 - Robustness check on number of latent states

tck = "AAPL" # Ticker label (or the prefix of the CSV)
train_period = ("2003-02-10", "2004-09-10") # Training Period
test_period = ("2004-09-13", "2005-01-21") # Testing Period
latency = 10
n_states = [3, 4, 5, 6]

mapes = {}
dpas = {}

for n in n_states:
    results, _ = run_trial(
        dfs, tck, train_period, test_period, latency=latency, n_states=n
    )
    mapes[n] = results["MAPE"]
    dpas[n] = results["DPA"]

In [ ]:
# Experiment 2 - Robustness check on context window size

tck = "AAPL" # Ticker label (or the prefix of the CSV)
train_period = ("2003-02-10", "2004-09-10") # Training Period
test_period = ("2004-09-13", "2005-01-21") # Testing Period
latency = [6, 10, 20, 50]
n_states = 4

mapes = {}
dpas = {}

for l in latency:
    results, _ = run_trial(
        dfs, tck, train_period, test_period, latency=l, n_states=n_states
    )
    mapes[l] = results["MAPE"]
    dpas[l] = results["DPA"]

In [7]:
# Experiment 3 - Generalizability test on different stocks at same time period

tcks = ["FORD", "MACYS", "SP500"] # Ticker label (or the prefix of the CSV)
train_period = ("2003-02-10", "2004-09-10") # Training Period
test_period = ("2004-09-13", "2005-01-21") # Testing Period
latency = 10
n_states = 4

for tck in tcks:
    run_trial(
        dfs, tck, train_period, test_period, latency=latency, n_states=n_states
    )

Training HMM on train period ('2003-02-10', '2004-09-10')
Training Complete
Generating Predictions for test period ('2004-09-13', '2005-01-21')


100%|██████████| 92/92 [00:00<00:00, 2097152.00it/s]


Predictions Generated
FORD trial with latent states = 4, context window size = 10
MAPE = 0.019786732330045825, DPA = 0.45652173913043476
--------------------------------------------------------------------------------
Training HMM on train period ('2003-02-10', '2004-09-10')
Training Complete
Generating Predictions for test period ('2004-09-13', '2005-01-21')


100%|██████████| 92/92 [00:00<00:00, 2489522.37it/s]


Predictions Generated
MACYS trial with latent states = 4, context window size = 10
MAPE = 0.019642259760057762, DPA = 0.41304347826086957
--------------------------------------------------------------------------------
Training HMM on train period ('2003-02-10', '2004-09-10')
Training Complete
Generating Predictions for test period ('2004-09-13', '2005-01-21')


100%|██████████| 92/92 [00:00<00:00, 2489522.37it/s]


Predictions Generated
SP500 trial with latent states = 4, context window size = 10
MAPE = 0.005537558683297186, DPA = 0.45652173913043476
--------------------------------------------------------------------------------


In [8]:
# Experiment 4 - Generalizability test of same stock at different time periods

tck = "AAPL" # Ticker label (or the prefix of the CSV)
train_periods = [("2005-02-10", "2005-09-09"), 
                 ("2011-02-10", "2011-09-12"),
                 ("2015-02-10", "2015-09-10")] 
test_periods = [("2005-09-12", "2006-01-20"), 
                ("2011-09-13", "2012-01-23"),
                ("2015-09-14", "2016-01-21")] 
latency = 10
n_states = 4

for i in range(len(train_periods)):
    train_period = train_periods[i]
    test_period = test_periods[i]
    run_trial(
        dfs, tck, train_period, test_period, latency=latency, n_states=n_states
    )

Training HMM on train period ('2005-02-10', '2005-09-09')
Training Complete
Generating Predictions for test period ('2005-09-12', '2006-01-20')


100%|██████████| 91/91 [00:00<00:00, 2108738.48it/s]


Predictions Generated
AAPL trial with latent states = 4, context window size = 10
MAPE = 0.0156514928163111, DPA = 0.4725274725274725
--------------------------------------------------------------------------------
Training HMM on train period ('2011-02-10', '2011-09-12')


/opt/miniconda3/envs/py311/lib/python3.11/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/opt/miniconda3/envs/py311/lib/python3.11/site-packages/hmmlearn/hmm.py:809: RuntimeWarning: invalid value encountered in divide
  self.covars_ = c_n / c_d + self.min_covar


Training Complete
Generating Predictions for test period ('2011-09-13', '2012-01-23')


ValueError: startprob_ must sum to 1 (got nan)